# Analyze data with machine learning

Although a lot of interesting question could be addressed with this dataset, we will focus on one single use case, summarized by the following question: **Can we group the shared-bike station regarding their occupation chronicle?**

## Introduction

In [ ]:
from datetime import datetime, date, timedelta
import os
import pytz

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from sklearn.cluster import KMeans
from sqlalchemy import create_engine

## Configuration

In [ ]:
DATADIR = "../data"

In [ ]:
HOST = "localhost"
PORT = 5432
USER = "rde"
DBNAME = "jitenshea"

## Utilities

In [ ]:
def get_engine():
    url = "postgresql://{user}@{host}:{port}/{dbname}".format(user=USER, host=HOST, port=PORT, dbname=DBNAME)
    return create_engine(url)
engine = get_engine()

## Compute stations clusters

### Retrieve the previously stored timeseries data

In [ ]:
stop = date.today()
start = stop - timedelta(7)
start, stop

In [ ]:
availability_input_file = "{begin}-{end}.csv".format(begin=start.strftime("%Y%m%d"), end=stop.strftime("%Y%m%d"))
availability_input_path = os.path.join(DATADIR, "lyon", "history", availability_input_file)
availability_input_path

In [ ]:
df = pd.read_csv(availability_input_path, parse_dates=["timestamp"])

In [ ]:
df.shape

In [ ]:
df.timestamp.min(), df.timestamp.max()

In order to control the clustering process in a wider way, we may consider the time period as a input parameter, hence select the data accordingly.

We can note that timestamped data are stored with a timezone info, hence we must declare timezone-aware datetimes here.

In [ ]:
today = date.today()
stop = datetime(today.year, today.month, today.day, 0, 0, tzinfo=pytz.utc)
start = stop - timedelta(7)
start, stop

In [ ]:
df = df[(df["timestamp"] >= start) & (df["timestamp"] <= stop)]
df.shape

### Pré-traitement des données

First we remove from the analysis shared-bike stations that looks unused during the period (the station remained empty).

In [ ]:
df.columns = ["ts", "nb_bikes", "station_id"]

In [ ]:
max_bikes = df.groupby("station_id")["nb_bikes"].max()
unactive_stations = max_bikes[max_bikes==0].index.tolist()
df = df[~ df["station_id"].isin(unactive_stations)]

In [ ]:
unactive_stations, df.shape

Then we resample the data each 5 minutes and group it with respect to station IDs :

In [ ]:
rdf = (df.set_index("ts")
       .groupby("station_id")["nb_bikes"]
       .resample("5T")
       .mean()
       .bfill()
       .unstack(0))
rdf.shape

*Spoiler:* this data is already sampled into 5-minute periods, however we keep the code as is for data consistency.

Then we can remove the week-end days from analysis, by anticipating that they may "pollute" the cluster formation:

In [ ]:
rdf = rdf[rdf.index.weekday < 5]

And finally, we apply a naive normalization scheme to consider station filling rates instead of bike quantities:

In [ ]:
rdf = rdf / rdf.max()

Once the data is normalized, we can aggregate availability at hour level for clustering step:

In [ ]:
rdf["hour"] = rdf.index.hour
rdf = rdf.groupby("hour").mean()

This last operation provides the typical week day profile of each station, at each hour of the day:

In [ ]:
rdf.iloc[:,:3]

### Apply the k-mean algorithm

At this point, we have a clusterable dataset! Let apply the easiest step...

In [ ]:
N_CLUSTERS = 4

Here we are, `scikit-learn` makes it as easy as two lines of Python:

In [ ]:
model = KMeans(n_clusters=N_CLUSTERS, random_state=0)
model.fit(rdf.T)
model

In [ ]:
df_labels = pd.DataFrame({"station_id": rdf.columns, "cluster_id": model.labels_})
df_centroids = pd.DataFrame(model.cluster_centers_, columns=["h{:02d}".format(i) for i in range(24)]).reset_index()
df_centroids.columns.values[0] = "cluster_id"

For each station, we have a cluster...

In [ ]:
palette = sns.color_palette(n_colors=N_CLUSTERS)
sns.set(style="whitegrid")

In [ ]:
pie_labels = [": ".join([str(u), str(c)]) + " stations" for u, c in zip(df_labels.cluster_id.unique(), df_labels.cluster_id.value_counts())]
cluster_pie = plt.pie(labels=pie_labels, x=df_labels.cluster_id.value_counts(), colors=palette)

...and for each cluster, we have the centroid expressed as a typical week day profile!

In [ ]:
df_centroids.iloc[:,:10]

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(12, 4))
ax.set_ylim((0, 1))
centroid_plot = sns.lineplot(data=df_centroids.drop(columns=["cluster_id"]).T, ax=ax)

## Store the clustering outputs to the application database

We previously clustered the shared-bike stations and deduced typical week day profiles, the job is almost done for ending the loop: we still have to store this new produced data to the database.

As the produced outputs highly depends on the chosen time period, it is recommended to store the start and stop datetimes as well: this modelization choice will allow us to store multiple clustering outputs into the database.

In [ ]:
df_labels.loc[:, "start"] = start
df_labels.loc[:, "stop"] = stop
df_centroids.loc[:, "start"] = start
df_centroids.loc[:, "stop"] = stop
df_labels.head()

As a final transformation, we consider string-formatted station IDs:

In [ ]:
df_labels.loc[:, "station_id"] = df_labels["station_id"].astype(str)

In [ ]:
engine.execute("DROP TABLE IF EXISTS lyon.cluster;")
df_labels.to_sql("cluster", schema="lyon", con=engine, index=False)

In [ ]:
engine.execute("DROP TABLE IF EXISTS lyon.centroid;")
df_centroids.to_sql("centroid", schema="lyon", con=engine, index=False)

That's all folks, for populating the database!

Now the database contains:
- the shared-bike station description;
- one week of bike availability timeseries;
- the clustering outputs.